# A CNN for the fashion mnist dataset

#### Imports

In [148]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import load_model
from PIL import Image
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt

## Dataset

In [127]:
# load dataset
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
print('done')

done


In [128]:
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [129]:
# convert from integers to floats
trainX, testX = trainX.astype('float32'), testX.astype('float32')

#### Images of dataset

In [211]:
# find index for a specific class
def find_first_index(data, x):
    index_list = np.where(data == x)
    return index_list[0]

# show image for numpy array
def show_image(data, index):
    data = np.reshape(data[index], (data.shape[1], data.shape[2]))
    data = (data * 255).astype(np.uint8)
    img = Image.fromarray(data)
    img.show()

# choose class number (0-10) and item number (0-:)
# 0 = t-shirt/top
# 1 = pants
# 2 = pullover
# 3 = dress
# 4 = coat
# 5 = sandal
# 6 = shirt
# 7 = sneaker
# 8 = bag
# 9 = ankle shoes
class_number = 9
item_number = 0
index = find_first_index(trainY, class_number)[item_number]
show_image(trainX, index)

In [106]:
# normalize to range 0-1
trainX, testX = trainX / 255, testX / 255

In [107]:
# one hot encode target values
trainY, testY = to_categorical(trainY), to_categorical(testY)

## Model

In [34]:
# define model
model = Sequential()
model.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(2))
model.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D(2))
# model.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(200, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 26, 26, 128)       1280      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 11, 11, 128)       147584    
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 3200)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 200)               640200    
_________________________________________________________________
dense_31 (Dense)             (None, 10)              

In [35]:
# fit model
model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 54s - loss: 0.4118 - accuracy: 0.8506
Epoch 2/10
1875/1875 - 55s - loss: 0.2759 - accuracy: 0.8979
Epoch 3/10
1875/1875 - 55s - loss: 0.2308 - accuracy: 0.9145
Epoch 4/10
1875/1875 - 55s - loss: 0.1972 - accuracy: 0.9270
Epoch 5/10
1875/1875 - 56s - loss: 0.1705 - accuracy: 0.9361
Epoch 6/10
1875/1875 - 57s - loss: 0.1472 - accuracy: 0.9438
Epoch 7/10
1875/1875 - 56s - loss: 0.1245 - accuracy: 0.9532
Epoch 8/10
1875/1875 - 55s - loss: 0.1067 - accuracy: 0.9589
Epoch 9/10
1875/1875 - 56s - loss: 0.0905 - accuracy: 0.9653
Epoch 10/10
1875/1875 - 55s - loss: 0.0767 - accuracy: 0.9712


In [39]:
# evaluate model
loss, acc = model.evaluate(testX, testY, verbose=0)
print(loss, acc)

0.3928087651729584 0.9082000255584717


In [76]:
# save model
model.save('model_cnn_2layers_128.h5')

## Model loaded

In [79]:
# load model
model_loaded = load_model('model_cnn_2layers_128.h5')

In [80]:
# evaluate model again
model_loaded.evaluate(testX, testY, verbose=0)

[0.3928087651729584, 0.9082000255584717]

In [82]:
# predict test data
predictions = model_loaded.predict(testX)

In [226]:
# check image and prediction for specific index
i = 1000
show_image(testX, i)
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for prediction, class_ in zip(predictions[i], classes):
    print('%s - %.2f' % (class_, prediction*100))
print('\nActual label: %s' % classes[testY[i]])

T-shirt/top - 0.34
Trouser - 0.00
Pullover - 0.00
Dress - 0.00
Coat - 0.00
Sandal - 0.00
Shirt - 99.66
Sneaker - 0.00
Bag - 0.00
Ankle boot - 0.00

Actual label: T-shirt/top


In [229]:
print(*predictions[1000])

0.003423432 9.907594e-10 1.3374826e-07 1.5041659e-06 5.323102e-08 4.963958e-08 0.9965749 5.462862e-09 1.1404088e-10 1.0462664e-10
